In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings

warnings.filterwarnings('ignore')

In [23]:
# parameters
customer_id = '5'
formatted_attribute = 'material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """V-Belt"""

regex_pattern = r"(?i)(\s[^\.]\d*\.?\d* in\w*\s?O.D.)|([^\.]\d*\.\d* in\w*\s?OAL)|([^\.]\d*\-\d*\/\d* in\w*\s?O.D.)|([^\.]\d*\.\d in\w*\s?O.D.)"
# regex_pattern = r"(?i)(\b[8][1-9] in\w* Overall Length)|(\b[9][0-9] in\w* Overall Length)|(\b1[0][0] in\w* Overall Length)"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [24]:
metric = ' Inches'

range_string = """
 0 - 0.5000 Inches
 0 - 10 Inches
 0.5001 - 1.0000 Inches
 0 - 5 Inches
 1.0001 - 1.5000 Inches
 11 - 15 Inches
 11 - 20 Inches
 1.5001 - 2.0000 Inches
 16 - 20 Inches
 2.0001 - 2.5000 Inches
 21 - 25 Inches
 21 - 30 Inches
 2.5001 - 3.0000 Inches
 26 - 30 Inches
 3.0001 - 3.5000 Inches
 31 - 40 Inches
 3.5001 - 4.0000 Inches
 4.0001 - 4.5000 Inches
 41 - 50 Inches
 4.5001 - 5.0000 Inches
 6 - 10 Inches
 Greater Than 30 Inches
 Greater than 5.0000 Inches
 Greater than 50 Inches
 """

In [25]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [26]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, values]
Index: []

In [14]:
df[100:200]

Empty DataFrame
Columns: [attribute, value, customer_name, external_id, long_desc, resolution]
Index: []

In [15]:
wipe=df[['external_id']].astype(str)
wipe[curation_col]=''
wipe

Empty DataFrame
Columns: [external_id, Q:outside_diameter]
Index: []

In [50]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0 - 0.5000 Inches',
 '0 - 10 Inches',
 '0.5001 - 1.0000 Inches',
 '0 - 5 Inches',
 '1.0001 - 1.5000 Inches',
 '11 - 15 Inches',
 '11 - 20 Inches',
 '1.5001 - 2.0000 Inches',
 '16 - 20 Inches',
 '2.0001 - 2.5000 Inches',
 '21 - 25 Inches',
 '21 - 30 Inches',
 '2.5001 - 3.0000 Inches',
 '26 - 30 Inches',
 '3.0001 - 3.5000 Inches',
 '31 - 40 Inches',
 '3.5001 - 4.0000 Inches',
 '4.0001 - 4.5000 Inches',
 '41 - 50 Inches',
 '4.5001 - 5.0000 Inches',
 '6 - 10 Inches',
 'Greater Than 30 Inches',
 'Greater than 5.0000 Inches',
 'Greater than 50 Inches']

In [51]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0        []
1        []
2        []
3        []
4        []
         ..
21681    []
21682    []
21683    []
21684    []
21685    []
Name: long_desc, Length: 21686, dtype: object

In [52]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [53]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [54]:
df[df['matches'].astype(str)=='[]']

attribute          value       customer_name external_id  \
44     outside_diameter            n/a  motionindustriesus    00702650   
53     outside_diameter            n/a  motionindustriesus    00703100   
54     outside_diameter            n/a  motionindustriesus    00703106   
55     outside_diameter            n/a  motionindustriesus    00703112   
56     outside_diameter            n/a  motionindustriesus    00703118   
...                 ...            ...                 ...         ...   
21612  outside_diameter  0 - 10 Inches  motionindustriesus    08709311   
21613  outside_diameter  0 - 10 Inches  motionindustriesus    08709312   
21614  outside_diameter  0 - 10 Inches  motionindustriesus    08709314   
21615  outside_diameter  0 - 10 Inches  motionindustriesus    08709315   
21616  outside_diameter  0 - 10 Inches  motionindustriesus    08709318   

                                                                                                                                      long_desc  \
44                Bushing Bore V-Belt Pulley - D Belt Section, 4 Groove, 22.0000 in Pitch Dia., QD Bushed, J Series Bushing, Cast Iron Material   
53                                                                                                                                       Sheave   
54                                                                                                                                       Sheave   
55                                                                                                                                       Sheave   
56                                                                                                                                       Sheave   
...                                                                                                                                         ...   
21612     78104746 Variable Pitch Sheave - VP Series, 2 Groove, 4L/5L Belt Section Size, 1.0000 in Bore, 6.0000 in OD, Finished, Steel Material   
21613     78104754 Variable Pitch Sheave - VP Series, 2 Groove, 4L/5L Belt Section Size, 1.6250 in Bore, 6.0000 in OD, Finished, Steel Material   
21614  78104843 Variable Pitch Sheave - VP Series, 2 Groove, 3L/4L/5L Belt Section Size, 0.7500 in Bore, 6.5500 in OD, Finished, Steel Material   
21615  78104854 Variable Pitch Sheave - VP Series, 2 Groove, 3L/4L/5L Belt Section Size, 1.6250 in Bore, 6.5500 in OD, Finished, Steel Material   
21616     78104894 Variable Pitch Sheave - VP Series, 2 Groove, 4L/5L Belt Section Size, 1.6250 in Bore, 7.1000 in OD, Finished, Steel Material   

      resolution matches  
44      standard      []  
53      standard      []  
54      standard      []  
55      standard      []  
56      standard      []  
...          ...     ...  
21612   standard      []  
21613   standard      []  
21614   standard      []  
21615   standard      []  
21616   standard      []  

[8154 rows x 7 columns]

In [61]:
x[x['value'].astype(str)!='Greater than 30 Inches'][100:150]

attribute                       value       customer_name  \
5799  outside_diameter                         n/a  motionindustriesus   
5800  outside_diameter                         n/a  motionindustriesus   
5801  outside_diameter                         n/a  motionindustriesus   
5802  outside_diameter                         n/a  motionindustriesus   
5803  outside_diameter                         n/a  motionindustriesus   
5804  outside_diameter                         n/a  motionindustriesus   
5951  outside_diameter      Greater than 50 Inches  motionindustriesus   
5952  outside_diameter      Greater than 50 Inches  motionindustriesus   
5953  outside_diameter      Greater than 50 Inches  motionindustriesus   
5954  outside_diameter      Greater than 50 Inches  motionindustriesus   
5955  outside_diameter      Greater than 50 Inches  motionindustriesus   
5956  outside_diameter      Greater than 50 Inches  motionindustriesus   
5957  outside_diameter      Greater than 50 Inches  motionindustriesus   
5958  outside_diameter      Greater than 50 Inches  motionindustriesus   
5959  outside_diameter      Greater than 50 Inches  motionindustriesus   
5960  outside_diameter      Greater than 50 Inches  motionindustriesus   
6126  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6127  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6128  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6129  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6130  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6131  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6132  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6133  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6134  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6147  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6149  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6150  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6151  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6152  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6154  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6155  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6156  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6157  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6158  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6159  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6160  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6161  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6162  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6163  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6164  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6165  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6166  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6167  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6168  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6169  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6170  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6171  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6172  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   
6173  outside_diameter  Greater than 5.0000 Inches  motionindustriesus   

     external_id  \
5799    08710053   
5800    08710054   
5801    08710055   
5802    08710056   
5803    08710057   
5804    08710058   
5951    00701659   
5952    00701827   
5953    00701987   
5954    00702148   
5955    00702300  

In [11]:
df[df['external_id'].astype(str)=='00739441']

attribute           value       customer_name  \
2633  micro_v_belt_overall_length  61 - 80 Inches  motionindustriesus   

     external_id  \
2633    00739441   

                                                                                         long_desc  \
2633  FleetRunner® Micro-V Belt - K Section, 4 Ribs, 0.560 in Width, 42.62 in OAL, Standard Series   

     resolution         matches  
2633      rules  [42.62 in OAL]

In [63]:
# x['matches'].explode().value_counts()

In [64]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [65]:
split_col(x)

In [66]:
len(x['matches'].explode().value_counts())

562

In [16]:
# error

In [67]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
x

attribute          value       customer_name external_id  \
0      outside_diameter            n/a  motionindustriesus    00700013   
1      outside_diameter            n/a  motionindustriesus    00700026   
2      outside_diameter            n/a  motionindustriesus    00700287   
3      outside_diameter            n/a  motionindustriesus    00700301   
4      outside_diameter            n/a  motionindustriesus    00700551   
...                 ...            ...                 ...         ...   
21681  outside_diameter  0 - 10 Inches  motionindustriesus    08710046   
21682  outside_diameter  0 - 10 Inches  motionindustriesus    08710047   
21683  outside_diameter  0 - 10 Inches  motionindustriesus    08710048   
21684  outside_diameter  0 - 10 Inches  motionindustriesus    08710049   
21685  outside_diameter  0 - 10 Inches  motionindustriesus    08710050   

                                                                                                                                                long_desc  \
0      7832-2200 Sheave - A, B Belt Section, 2 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
1      7832-2300 Sheave - A, B Belt Section, 2 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
2      7832-3200 Sheave - A, B Belt Section, 3 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
3      7832-3300 Sheave - A, B Belt Section, 3 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
4      7832-4200 Sheave - A, B Belt Section, 4 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
...                                                                                                                                                   ...   
21681                                                                                7963-0003 Adjustable Speed Sheave - 9.56 in O.D., 1.625 in Bore Size   
21682                                                                                7963-0004 Adjustable Speed Sheave - 9.56 in O.D., 1.875 in Bore Size   
21683                                                                                7963-0005 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21684                                                                                7963-0006 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21685                                                                                7963-0007 Adjustable Speed Sheave - 9.56 in O.D., 2.375 in Bore Size   

      resolution matches  
0       standard   20.35  
1       standard   30.35  
2       standard   20.35  
3       standard   30.35  
4       standard   20.35  
...          ...     ...  
21681   standard    9.56  
21682   standard    9.56  
21683   standard    9.56  
21684   standard    9.56  
21685   standard    9.56  

[13532 rows x 7 columns]

In [68]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [69]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

attribute          value       customer_name external_id  \
0      outside_diameter            n/a  motionindustriesus    00700013   
1      outside_diameter            n/a  motionindustriesus    00700026   
2      outside_diameter            n/a  motionindustriesus    00700287   
3      outside_diameter            n/a  motionindustriesus    00700301   
4      outside_diameter            n/a  motionindustriesus    00700551   
...                 ...            ...                 ...         ...   
21681  outside_diameter  0 - 10 Inches  motionindustriesus    08710046   
21682  outside_diameter  0 - 10 Inches  motionindustriesus    08710047   
21683  outside_diameter  0 - 10 Inches  motionindustriesus    08710048   
21684  outside_diameter  0 - 10 Inches  motionindustriesus    08710049   
21685  outside_diameter  0 - 10 Inches  motionindustriesus    08710050   

                                                                                                                                                long_desc  \
0      7832-2200 Sheave - A, B Belt Section, 2 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
1      7832-2300 Sheave - A, B Belt Section, 2 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
2      7832-3200 Sheave - A, B Belt Section, 3 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
3      7832-3300 Sheave - A, B Belt Section, 3 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
4      7832-4200 Sheave - A, B Belt Section, 4 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
...                                                                                                                                                   ...   
21681                                                                                7963-0003 Adjustable Speed Sheave - 9.56 in O.D., 1.625 in Bore Size   
21682                                                                                7963-0004 Adjustable Speed Sheave - 9.56 in O.D., 1.875 in Bore Size   
21683                                                                                7963-0005 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21684                                                                                7963-0006 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21685                                                                                7963-0007 Adjustable Speed Sheave - 9.56 in O.D., 2.375 in Bore Size   

      resolution matches  
0       standard   20.35  
1       standard   30.35  
2       standard   20.35  
3       standard   30.35  
4       standard   20.35  
...          ...     ...  
21681   standard    9.56  
21682   standard    9.56  
21683   standard    9.56  
21684   standard    9.56  
21685   standard    9.56  

[13532 rows x 7 columns]

In [70]:
x['matches']=x['matches'].astype(float)

In [71]:
x

attribute          value       customer_name external_id  \
0      outside_diameter            n/a  motionindustriesus    00700013   
1      outside_diameter            n/a  motionindustriesus    00700026   
2      outside_diameter            n/a  motionindustriesus    00700287   
3      outside_diameter            n/a  motionindustriesus    00700301   
4      outside_diameter            n/a  motionindustriesus    00700551   
...                 ...            ...                 ...         ...   
21681  outside_diameter  0 - 10 Inches  motionindustriesus    08710046   
21682  outside_diameter  0 - 10 Inches  motionindustriesus    08710047   
21683  outside_diameter  0 - 10 Inches  motionindustriesus    08710048   
21684  outside_diameter  0 - 10 Inches  motionindustriesus    08710049   
21685  outside_diameter  0 - 10 Inches  motionindustriesus    08710050   

                                                                                                                                                long_desc  \
0      7832-2200 Sheave - A, B Belt Section, 2 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
1      7832-2300 Sheave - A, B Belt Section, 2 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
2      7832-3200 Sheave - A, B Belt Section, 3 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
3      7832-3300 Sheave - A, B Belt Section, 3 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
4      7832-4200 Sheave - A, B Belt Section, 4 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
...                                                                                                                                                   ...   
21681                                                                                7963-0003 Adjustable Speed Sheave - 9.56 in O.D., 1.625 in Bore Size   
21682                                                                                7963-0004 Adjustable Speed Sheave - 9.56 in O.D., 1.875 in Bore Size   
21683                                                                                7963-0005 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21684                                                                                7963-0006 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21685                                                                                7963-0007 Adjustable Speed Sheave - 9.56 in O.D., 2.375 in Bore Size   

      resolution  matches  
0       standard    20.35  
1       standard    30.35  
2       standard    20.35  
3       standard    30.35  
4       standard    20.35  
...          ...      ...  
21681   standard     9.56  
21682   standard     9.56  
21683   standard     9.56  
21684   standard     9.56  
21685   standard     9.56  

[13532 rows x 7 columns]

# matches

In [72]:
matches=x
matches

attribute          value       customer_name external_id  \
0      outside_diameter            n/a  motionindustriesus    00700013   
1      outside_diameter            n/a  motionindustriesus    00700026   
2      outside_diameter            n/a  motionindustriesus    00700287   
3      outside_diameter            n/a  motionindustriesus    00700301   
4      outside_diameter            n/a  motionindustriesus    00700551   
...                 ...            ...                 ...         ...   
21681  outside_diameter  0 - 10 Inches  motionindustriesus    08710046   
21682  outside_diameter  0 - 10 Inches  motionindustriesus    08710047   
21683  outside_diameter  0 - 10 Inches  motionindustriesus    08710048   
21684  outside_diameter  0 - 10 Inches  motionindustriesus    08710049   
21685  outside_diameter  0 - 10 Inches  motionindustriesus    08710050   

                                                                                                                                                long_desc  \
0      7832-2200 Sheave - A, B Belt Section, 2 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
1      7832-2300 Sheave - A, B Belt Section, 2 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
2      7832-3200 Sheave - A, B Belt Section, 3 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
3      7832-3300 Sheave - A, B Belt Section, 3 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
4      7832-4200 Sheave - A, B Belt Section, 4 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
...                                                                                                                                                   ...   
21681                                                                                7963-0003 Adjustable Speed Sheave - 9.56 in O.D., 1.625 in Bore Size   
21682                                                                                7963-0004 Adjustable Speed Sheave - 9.56 in O.D., 1.875 in Bore Size   
21683                                                                                7963-0005 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21684                                                                                7963-0006 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21685                                                                                7963-0007 Adjustable Speed Sheave - 9.56 in O.D., 2.375 in Bore Size   

      resolution  matches  
0       standard    20.35  
1       standard    30.35  
2       standard    20.35  
3       standard    30.35  
4       standard    20.35  
...          ...      ...  
21681   standard     9.56  
21682   standard     9.56  
21683   standard     9.56  
21684   standard     9.56  
21685   standard     9.56  

[13532 rows x 7 columns]

In [73]:
matches['matches']=matches['matches'].astype(float)

In [74]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  0 and num <= 0.5000:
        return " 0 - 0.5000 Inches"
        
    if num >=  0 and num <= 10:
        return " 0 - 10 Inches"
        
    if num >=  0.5001 and num <= 1.0000:
        return " 0.5001 - 1.0000 Inches"
        
    if num >=  0 and num <= 5:
        return " 0 - 5 Inches"
        
    if num >=  1.0001 and num <= 1.5000:
        return " 1.0001 - 1.5000 Inches"
        
    if num >=  11 and num <= 15:
        return " 11 - 15 Inches"
        
    if num >=  11 and num <= 20:
        return " 11 - 20 Inches"
        
    if num >=  1.5001 and num <= 2.0000:
        return " 1.5001 - 2.0000 Inches"
        
    if num >=  16 and num <= 20:
        return " 16 - 20 Inches"
        
    if num >=  2.0001 and num <= 2.5000:
        return " 2.0001 - 2.5000 Inches"
        
    if num >=  21 and num <= 25:
        return " 21 - 25 Inches"
        
    if num >=  21 and num <= 30:
        return " 21 - 30 Inches"
        
    if num >= 

In [91]:
matches[matches['value'].astype(str)=='Greater than 50 Inches']

attribute                   value       customer_name  \
5951  outside_diameter  Greater than 50 Inches  motionindustriesus   
5952  outside_diameter  Greater than 50 Inches  motionindustriesus   
5953  outside_diameter  Greater than 50 Inches  motionindustriesus   
5954  outside_diameter  Greater than 50 Inches  motionindustriesus   
5955  outside_diameter  Greater than 50 Inches  motionindustriesus   
5956  outside_diameter  Greater than 50 Inches  motionindustriesus   
5957  outside_diameter  Greater than 50 Inches  motionindustriesus   
5958  outside_diameter  Greater than 50 Inches  motionindustriesus   
5959  outside_diameter  Greater than 50 Inches  motionindustriesus   
5960  outside_diameter  Greater than 50 Inches  motionindustriesus   

     external_id  \
5951    00701659   
5952    00701827   
5953    00701987   
5954    00702148   
5955    00702300   
5956    00702448   
5957    00702571   
5958    00707735   
5959    00707842   
5960    00707953   

                                                                                                long_desc  \
5951    Sheave - C Belt Section, 3 Groove, 50.40 in O.D., QD Bushed, F Series Bushing, CAST IRON Material   
5952    Sheave - C Belt Section, 4 Groove, 50.40 in O.D., QD Bushed, J Series Bushing, CAST IRON Material   
5953    Sheave - C Belt Section, 5 Groove, 50.40 in O.D., QD Bushed, J Series Bushing, CAST IRON Material   
5954    Sheave - C Belt Section, 6 Groove, 50.40 in O.D., QD Bushed, M Series Bushing, CAST IRON Material   
5955    Sheave - C Belt Section, 8 Groove, 50.40 in O.D., QD Bushed, M Series Bushing, CAST IRON Material   
5956   Sheave - C Belt Section, 10 Groove, 50.40 in O.D., QD Bushed, M Series Bushing, CAST IRON Material   
5957   Sheave - C Belt Section, 12 Groove, 50.40 in O.D., QD Bushed, M Series Bushing, CAST IRON Material   
5958   Sheave - 8V Belt Section, 6 Groove, 53.00 in O.D., QD Bushed, N Series Bushing, CAST IRON Material   
5959   Sheave - 8V Belt Section, 8 Groove, 53.00 in O.D., QD Bushed, P Series Bushing, CAST IRON Material   
5960  Sheave - 8V Belt Section, 10 Groove, 53.00 in O.D., QD Bushed, P Series Bushing, CAST IRON Material   

     resolution  matches           Q:outside_diameter  
5951   standard     50.4   Greater than 5.0000 Inches  
5952   standard     50.4   Greater than 5.0000 Inches  
5953   standard     50.4   Greater than 5.0000 Inches  
5954   standard     50.4   Greater than 5.0000 Inches  
5955   standard     50.4   Greater than 5.0000 Inches  
5956   standard     50.4   Greater than 5.0000 Inches  
5957   standard     50.4   Greater than 5.0000 Inches  
5958   standard     53.0   Greater than 5.0000 Inches  
5959   standard     53.0   Greater than 5.0000 Inches  
5960   standard     53.0   Greater than 5.0000 Inches

In [75]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

attribute          value       customer_name external_id  \
0      outside_diameter            n/a  motionindustriesus    00700013   
1      outside_diameter            n/a  motionindustriesus    00700026   
2      outside_diameter            n/a  motionindustriesus    00700287   
3      outside_diameter            n/a  motionindustriesus    00700301   
4      outside_diameter            n/a  motionindustriesus    00700551   
...                 ...            ...                 ...         ...   
21681  outside_diameter  0 - 10 Inches  motionindustriesus    08710046   
21682  outside_diameter  0 - 10 Inches  motionindustriesus    08710047   
21683  outside_diameter  0 - 10 Inches  motionindustriesus    08710048   
21684  outside_diameter  0 - 10 Inches  motionindustriesus    08710049   
21685  outside_diameter  0 - 10 Inches  motionindustriesus    08710050   

                                                                                                                                                long_desc  \
0      7832-2200 Sheave - A, B Belt Section, 2 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
1      7832-2300 Sheave - A, B Belt Section, 2 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
2      7832-3200 Sheave - A, B Belt Section, 3 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
3      7832-3300 Sheave - A, B Belt Section, 3 Groove, 29.6 (A) in, 30 (B) in Pitch Dia., 30.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
4      7832-4200 Sheave - A, B Belt Section, 4 Groove, 19.6 (A) in, 20 (B) in Pitch Dia., 20.35 in O.D., QD Bushed, SF Series Bushing, Cast Iron Material   
...                                                                                                                                                   ...   
21681                                                                                7963-0003 Adjustable Speed Sheave - 9.56 in O.D., 1.625 in Bore Size   
21682                                                                                7963-0004 Adjustable Speed Sheave - 9.56 in O.D., 1.875 in Bore Size   
21683                                                                                7963-0005 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21684                                                                                7963-0006 Adjustable Speed Sheave - 9.56 in O.D., 2.125 in Bore Size   
21685                                                                                7963-0007 Adjustable Speed Sheave - 9.56 in O.D., 2.375 in Bore Size   

      resolution  matches           Q:outside_diameter  
0       standard    20.35   Greater than 5.0000 Inches  
1       standard    30.35   Greater than 5.0000 Inches  
2       standard    20.35   Greater than 5.0000 Inches  
3       standard    30.35   Greater than 5.0000 Inches  
4       standard    20.35   Greater than 5.0000 Inches  
...          ...      ...                          ...  
21681   standard     9.56                0 - 10 Inches  
21682   standard     9.56                0 - 10 Inches  
21683   standard     9.56                0 - 10 Inches  
21684   standard     9.56                0 - 10 Inches  
21685   standard     9.56                0 - 10 Inches  

[13532 rows x 8 columns]

In [78]:
matches['Q:outside_diameter'].explode().value_counts()

 0 - 10 Inches                 8139
 11 - 15 Inches                1843
 11 - 20 Inches                1215
 Greater than 5.0000 Inches    1097
 21 - 25 Inches                 426
 31 - 40 Inches                 376
 21 - 30 Inches                 262
 41 - 50 Inches                 174
Name: Q:outside_diameter, dtype: int64

# fixed_na

In [79]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [80]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [81]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

Empty DataFrame
Columns: [attribute, value, customer_name, external_id, long_desc, resolution, matches, Q:outside_diameter]
Index: []

In [82]:
fixed_na['Q:outside_diameter'].explode().value_counts()

Series([], Name: Q:outside_diameter, dtype: int64)

# Wipe

In [83]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

external_id Q:outside_diameter
0        00700013                   
1        00700026                   
2        00700287                   
3        00700301                   
4        00700551                   
...           ...                ...
21681    08710046                   
21682    08710047                   
21683    08710048                   
21684    08710049                   
21685    08710050                   

[21686 rows x 2 columns]

# Null Values

In [62]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [98]:
na_external_id

attribute value       customer_name external_id  \
0     micro_v_belt_overall_length   n/a  motionindustriesus    00456606   
1     micro_v_belt_overall_length   n/a  motionindustriesus    00714493   
2     micro_v_belt_overall_length   n/a  motionindustriesus    00720491   
3     micro_v_belt_overall_length   n/a  motionindustriesus    00721316   
4     micro_v_belt_overall_length   n/a  motionindustriesus    00721323   
...                           ...   ...                 ...         ...   
2462  micro_v_belt_overall_length   n/a  motionindustriesus    08837833   
2463  micro_v_belt_overall_length   n/a  motionindustriesus    08837834   
2464  micro_v_belt_overall_length   n/a  motionindustriesus    08837835   
2465  micro_v_belt_overall_length   n/a  motionindustriesus    10538904   
2466  micro_v_belt_overall_length   n/a  motionindustriesus    10539054   

                                                                                                                        long_desc  \
0     Micro-V® 917430114 Micro V-Belt - 14 Sections, M Size, 301.00 in Effective Length, 0.3750 in Rib Width, 5.2500 in Top Width   
1       Micro-V® 917102208 Micro V-Belt - 8 Sections, J Size, 22.00 in Effective Length, 0.0938 in Rib Width, 0.7500 in Top Width   
2      Micro-V® 917104024 Micro V-Belt - 24 Sections, J Size, 40.00 in Effective Length, 0.0938 in Rib Width, 2.2512 in Top Width   
3      Micro-V® 9171-01804 Micro V-Belt - 4 Sections, J Size, 18.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
4       Micro-V® 917101904 Micro V-Belt - 4 Sections, J Size, 19.00 in Effective Length, 0.0938 in Rib Width, 0.3750 in Top Width   
...                                                                                                                           ...   
2462                                                                                                                 Micro-V Belt   
2463                                                                                                                 Micro-V Belt   
2464                                                                                                                 Micro-V Belt   
2465                                                                                                               Micro-V Belt -   
2466                                                                                                               Micro-V Belt -   

     resolution matches  
0          bulk      []  
1          bulk      []  
2          bulk      []  
3          bulk      []  
4          bulk      []  
...         ...     ...  
2462       bulk      []  
2463       bulk      []  
2464       bulk      []  
2465       bulk      []  
2466       bulk      []  

[2467 rows x 7 columns]

In [ ]:
error

In [65]:
gc = gcloud.Gsheets()

In [67]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [8]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [9]:
looks_good('Motion Industries', attribute, df, wipe)